# CBP
Retrieves data from the Census Bureau's County Business Patterns API.

In [8]:
# https://www.census.gov/programs-surveys/cbp/data.html
# Importing necessary packages.
import zipfile as zip
import pandas as pd
import csv
import os
import pyarrow.feather as feather

pd.set_option('display.max_columns', None)

# Set up file path:
from google.colab import drive
drive.mount('/content/gdrive')
%cd "/content/gdrive/My Drive/census-api"
target = os.getcwd()

import requests, sqlite3, json
from IPython.display import clear_output

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/census-api


In [9]:
%cd "/content/gdrive/My Drive/census-api"

/content/gdrive/My Drive/census-api


## API Call and Processing
Request the CBP data from the Census Bureau

In [10]:
api_key='1f46c23b38359f7b445a7d043208d4332b2bd8ec'
year='2019'
dsource='cbp'
ecols='EMP,PAYANN'
base_url = f'https://api.census.gov/data/'
base_url
#Dump files for api data storage
ejsonpath=os.path.join(target,'emp_data.json')

### CBP Employment Data
This data can be requested in a single api call - do not rerun the requests block if retrieval is successful but subsequent notebook changes are needed. Proceed to the next block and pull data from json dump file.

#### ***THIS BLOCK IS A REQUESTS BLOCK!*** 

In [ ]:
#Code 200 = success, do not rerun this block unless it's necessary
edata_url = f'{base_url}{year}/cbp?get={ecols}&for=county:*'
response=requests.get(edata_url)
if response.status_code==200:
    emp_data=response.json()
    with open(ejsonpath, 'w') as f:
        json.dump(emp_data, f)
    print('Data dumped to json file')
else:
    print('Problem with retrieval, response code',response.status_code)

Data dumped to json file


In [ ]:
with open(ejsonpath, 'r') as f:
    ejsondata=json.load(f)
type(ejsondata)
cbpemp=pd.DataFrame(ejsondata[1:], columns=ejsondata[0])
cbpemp['YEAR'] = year
display(cbpemp)

,EMP,PAYANN,state,county,YEAR
0,9783,363192,47,177,2019
1,55010,2240133,47,179,2019
2,8250,272872,47,183,2019
3,3287,98812,47,005,2019
4,820,28272,47,007,2019
...,...,...,...,...,...
3239,7582,332824,27,091,2019
3240,13809,703041,27,099,2019
3241,8959,391350,27,105,2019
3242,6927,209326,27,115,2019


In [13]:
for year in range(1986,2021):
  print(year)
  #Code 200 = success, do not rerun this block unless it's necessary
  edata_url = f'{base_url}{year}/cbp?get={ecols}&for=county:*'
  response=requests.get(edata_url)
  if response.status_code==200:
      emp_data=response.json()
      with open(ejsonpath, 'w') as f:
          json.dump(emp_data, f)
      print('Data dumped to json file')
  else:
      print('Problem with retrieval, response code',response.status_code)
      break
  with open(ejsonpath, 'r') as f:
      ejsondata=json.load(f)
  cbpemp=pd.DataFrame(ejsondata[1:], columns=ejsondata[0])
  cbpemp['YEAR'] = year
  if year==1986:
    df=cbpemp
  else:
    df=df.append(cbpemp)

1986
Data dumped to json file
1987
Data dumped to json file
1988
Data dumped to json file
1989
Data dumped to json file
1990
Data dumped to json file
1991
Data dumped to json file
1992
Data dumped to json file
1993
Data dumped to json file
1994
Data dumped to json file
1995


ConnectionError: ignored

In [7]:
df.sort_values(by=['state','county','YEAR'], inplace=True)
display(df)
df.to_stata('cbp.dta', write_index=False)  

,EMP,PAYANN,state,county,YEAR
2828,11265,385785,01,001,2020
3018,66943,2473578,01,003,2020
2819,6972,246231,01,005,2020
2820,4045,169424,01,007,2020
3032,6764,244398,01,009,2020
...,...,...,...,...,...
1234,1619,25891,72,151,2020
1045,3629,76076,72,153,2020
964,13931,676256,78,010,2020
1091,1911,66143,78,020,2020
